In [1]:
import math
import fractions
import anti_lib
from anti_lib import Vec
import numpy as np

In [2]:
#https://github.com/antiprism/antiprism_python/blob/master/anti_lib_progs/geodesic.py
    
def get_octahedron(verts, faces):
    """Return an octahedron"""
    X = 0.25 * math.sqrt(2)
    verts.extend([Vec(0.0, 0.5, 0.0), Vec(X, 0.0, -X),
                  Vec(X, 0.0, X), Vec(-X, 0.0, X),
                  Vec(-X, 0.0, -X), Vec(0.0, -0.5, 0.0)])

    faces.extend([(0, 1, 2), (0, 2, 3), (0, 3, 4), (0, 4, 1),
                  (5, 2, 1), (2, 5, 3), (3, 5, 4), (4, 5, 1)])


def get_tetrahedron(verts, faces):
    """Return an tetrahedron"""
    X = 1 / math.sqrt(3)
    verts.extend([Vec(-X, X, -X), Vec(-X, -X, X),
                  Vec(X, X, X), Vec(X, -X, -X)])
    faces.extend([(0, 1, 2), (0, 3, 1), (0, 2, 3), (2, 1, 3)])


def get_ico_coords():
    """Return icosahedron coordinate values"""
    phi = (math.sqrt(5) + 1) / 2
    rad = math.sqrt(phi+2)
    return 1/rad, phi/rad


def get_triangle(verts, faces):
    """Return an triangle"""
    if 1:
        Y = math.sqrt(3.0) / 12.0
        Z = -0.8
        verts.extend([Vec(-0.25, -Y, Z), Vec(0.25, -Y, Z),
                      Vec(0.0, 2 * Y, Z)])
        faces.extend([(0, 1, 2)])
    else:
        X, Z = get_ico_coords()
        verts.extend([Vec(-X, 0.0, -Z), Vec(X, 0.0, -Z),
                      Vec(0.0, Z, -X), Vec(0.0, -Z, -X)])
        faces.extend([(0, 1, 2), (0, 3, 1)])


def get_icosahedron(verts, faces):
    """Return an icosahedron"""
    X, Z = get_ico_coords()
    verts.extend([Vec(-X, 0.0, Z), Vec(X, 0.0, Z), Vec(-X, 0.0, -Z),
                  Vec(X, 0.0, -Z), Vec(0.0, Z, X), Vec(0.0, Z, -X),
                  Vec(0.0, -Z, X), Vec(0.0, -Z, -X), Vec(Z, X, 0.0),
                  Vec(-Z, X, 0.0), Vec(Z, -X, 0.0), Vec(-Z, -X, 0.0)])

    faces.extend([(0, 4, 1), (0, 9, 4), (9, 5, 4), (4, 5, 8), (4, 8, 1),
                  (8, 10, 1), (8, 3, 10), (5, 3, 8), (5, 2, 3), (2, 7, 3),
                  (7, 10, 3), (7, 6, 10), (7, 11, 6), (11, 0, 6), (0, 1, 6),
                  (6, 1, 10), (9, 0, 11), (9, 11, 2), (9, 2, 5), (7, 2, 11)])


def get_poly(poly, verts, edges, faces):
    """Return the base polyhedron"""
    if poly == 'i':
        get_icosahedron(verts, faces)
    elif poly == 'o':
        get_octahedron(verts, faces)
    elif poly == 't':
        get_tetrahedron(verts, faces)
    elif poly == 'T':
        get_triangle(verts, faces)
    else:
        return 0

    for face in faces:
        for i in range(0, len(face)):
            i2 = i + 1
            if(i2 == len(face)):
                i2 = 0

            if face[i] < face[i2]:
                edges[(face[i], face[i2])] = 0
            else:
                edges[(face[i2], face[i])] = 0

    return 1


def grid_to_points(grid, freq, div_by_len, f_verts, face):
    """Convert grid coordinates to Cartesian coordinates"""
    points = []
    v = []
    for vtx in range(3):
        v.append([Vec(0.0, 0.0, 0.0)])
        edge_vec = f_verts[(vtx + 1) % 3] - f_verts[vtx]
        if div_by_len:
            for i in range(1, freq + 1):
                v[vtx].append(edge_vec * float(i) / freq)
        else:
            ang = 2 * math.asin(edge_vec.mag() / 2.0)
            unit_edge_vec = edge_vec.unit()
            for i in range(1, freq + 1):
                len = math.sin(i * ang / freq) / \
                    math.sin(math.pi / 2 + ang / 2 - i * ang / freq)
                v[vtx].append(unit_edge_vec * len)

    for (i, j) in grid.values():

        if (i == 0) + (j == 0) + (i + j == freq) == 2:   # skip vertex
            continue
        # skip edges in one direction
        if (i == 0 and face[2] > face[0]) or (
                j == 0 and face[0] > face[1]) or (
                i + j == freq and face[1] > face[2]):
            continue

        n = [i, j, freq - i - j]
        v_delta = (v[0][n[0]] + v[(0-1) % 3][freq - n[(0+1) % 3]] -
                   v[(0-1) % 3][freq])
        pt = f_verts[0] + v_delta
        if not div_by_len:
            for k in [1, 2]:
                v_delta = (v[k][n[k]] + v[(k-1) % 3][freq - n[(k+1) % 3]] -
                           v[(k-1) % 3][freq])
                pt = pt + f_verts[k] + v_delta
            pt = pt / 3
        points.append(pt)

    return points


def make_grid(freq, m, n):
    """Make the geodesic pattern grid"""
    grid = {}
    rng = (2 * freq) // (m + n)
    for i in range(rng):
        for j in range(rng):
            x = i * (-n) + j * (m + n)
            y = i * (m + n) + j * (-m)

            if x >= 0 and y >= 0 and x + y <= freq:
                grid[(i, j)] = (x, y)

    return grid


def class_type(val_str):
    """Read the class pattern specifier"""
    order = ['first', 'second']
    num_parts = val_str.count(',')+1
    vals = val_str.split(',', 2)
    if num_parts == 1:
        if vals[0] == '1':
            pat = [1, 0, 1]
        elif vals[0] == '2':
            pat = [1, 1, 1]
        else:
            raise argparse.ArgumentTypeError(
                'class type can only be 1 or 2 when a single value is given')

    elif num_parts == 2:
        pat = []
        for i, num_str in enumerate(vals):
            try:
                num = int(num_str)
            except:
                raise argparse.ArgumentTypeError(
                    order[i] + ' class pattern value not an integer')
            if num < 0:
                raise argparse.ArgumentTypeError(
                    order[i] + ' class pattern cannot be negative')
            if num == 0 and i == 1 and pat[0] == 0:
                raise argparse.ArgumentTypeError(
                    ' class pattern values cannot both be 0')
            pat.append(num)

        rep = math.gcd(*pat)
        pat = [pat_num//rep for pat_num in pat]
        pat.append(rep)

    else:
        raise argparse.ArgumentTypeError(
            'class type contains more than two values')

    return pat


In [3]:
def geo(repeats=1, polyhedron="i", class_pattern=[1, 0, 1], flat_faced=False, equal_length=True, ):
    verts = []
    edges = {}
    faces = []
    get_poly(polyhedron, verts, edges, faces)

    (M, N, reps) = class_pattern
    repeats = repeats * reps
    freq = repeats * (M**2 + M*N + N**2)

    grid = {}
    grid = make_grid(freq, M, N)

    points = verts
    for face in faces:
        if polyhedron == 'T':
            face_edges = (0, 0, 0)  # generate points for all edges
        else:
            face_edges = face
        points[len(points):len(points)] = grid_to_points(
            grid, freq, equal_length,
            [verts[face[i]] for i in range(3)], face_edges)

    if not flat_faced:
        points = [p.unit() for p in points]  # Project onto sphere

    return points

In [27]:
ack = geo(class_pattern=[23,0,1])

In [28]:
ack

[Vec(-0.5257311121191336, 0.0, 0.85065080835204),
 Vec(0.5257311121191336, 0.0, 0.85065080835204),
 Vec(-0.5257311121191336, 0.0, -0.85065080835204),
 Vec(0.5257311121191336, 0.0, -0.85065080835204),
 Vec(0.0, 0.85065080835204, 0.5257311121191336),
 Vec(0.0, 0.85065080835204, -0.5257311121191336),
 Vec(0.0, -0.85065080835204, 0.5257311121191336),
 Vec(0.0, -0.85065080835204, -0.5257311121191336),
 Vec(0.85065080835204, 0.5257311121191336, 0.0),
 Vec(-0.85065080835204, 0.5257311121191336, 0.0),
 Vec(0.85065080835204, -0.5257311121191336, 0.0),
 Vec(-0.85065080835204, -0.5257311121191336, 0.0),
 Vec(-0.5148484764309716, 0.03786556063279143, 0.8564445373955465),
 Vec(-0.4791962405165047, 0.03876779022184371, 0.8768511969063869),
 Vec(-0.5025793493621171, 0.07744671136302034, 0.8610551692509627),
 Vec(-0.44100430576121646, 0.03964221977259441, 0.8966290742060334),
 Vec(-0.46550454497017285, 0.07928443954518882, 0.8814870936423955),
 Vec(-0.48883248016814557, 0.11864213515754529, 0.86427243

In [29]:
len(ack)

5292

In [39]:
ack = geo(class_pattern=[22,0,1])
len(ack)

4842

In [7]:
# https://github.com/antiprism/antiprism_python/blob/master/anti_lib_progs/sph_spiral.py

def angle_to_point(a, number_turns):
    a2 = 2 * a * number_turns   # angle turned around y axis
    r = math.sin(a)             # distance from y axis
    y = math.cos(a)
    x = r * math.sin(a2)
    z = r * math.cos(a2)
    return Vec(x, y, z)


# binary search for angle with distance rad from point with angle a0
def psearch(a1_delt, a1, a0, rad, number_turns):
    a_test = a1_delt + (a1 - a1_delt) / 2.0
    dist = (angle_to_point(a_test, number_turns) -
            angle_to_point(a0, number_turns)).mag()
    eps = 1e-5
    if rad + eps > dist > rad - eps:
        return a_test
    elif rad < dist:      # Search in first interval
        return psearch(a1_delt, a_test, a0, rad, number_turns)
    else:                 # Search in second interval
        return psearch(a_test, a1, a0, rad, number_turns)


def calc_points(number_turns=10, distance_between_points=None):
    points = []
    number_turns = number_turns
    if not number_turns:
        number_turns = 1e-12
    if distance_between_points:
        rad = 2*distance_between_points
    else:
        # half distance between turns on a rad 1 sphere
        rad = 2*math.sqrt(1 - math.cos(math.pi/(number_turns-1)))
    a0 = 0
    cur_point = Vec(0.0, 1.0, 0.0)
    points.append(cur_point)

    delt = math.atan(rad / 2) / 10
    a1 = a0 + .0999999 * delt                        # still within sphere
    while a1 < math.pi:
        if (cur_point - angle_to_point(a1, number_turns)).mag() > rad:
            a0 = psearch(a1 - delt, a1, a0, rad, number_turns)
            cur_point = angle_to_point(a0, number_turns)
            points.append(cur_point)
            a1 = a0

        a1 += delt

    return points

In [14]:
p = calc_points(number_turns=30, distance_between_points=np.radians(3.))

In [15]:
p

[Vec(0.0, 1.0, 0.0),
 Vec(0.00024672917235029224, 0.9945177839805889, 0.10456728202958271),
 Vec(0.09872161258767789, 0.9927205375057555, 0.06899259106869547),
 Vec(0.13078108902973962, 0.990936092584234, -0.03069148360841874),
 Vec(0.08073477790337984, 0.9891595625604537, -0.1226591025244517),
 Vec(-0.018077897322615662, 0.9873881797809957, -0.15728245947074015),
 Vec(-0.11672395261235746, 0.9856204327242787, -0.12217970978420964),
 Vec(-0.1754161062418291, 0.9838553731869009, -0.035465396123817),
 Vec(-0.17521636428566414, 0.9820926532469507, 0.06923327324977507),
 Vec(-0.11891481423664901, 0.980331596766365, 0.15750945157853935),
 Vec(-0.02526079531752953, 0.9785717329175814, 0.20435081539990912),
 Vec(0.0793055293349034, 0.9768131243006192, 0.19886365482655183),
 Vec(0.16866671211142387, 0.9750555134066304, 0.14428543239306704),
 Vec(0.2229219557877557, 0.9732987781295642, 0.05472924373001358),
 Vec(0.23160829010458497, 0.9715427309927387, -0.04962178765422791),
 Vec(0.194331930537

In [16]:
len(p)

1142